In [1]:
import pandas as pd

In [2]:
areas = ['City of London', 'Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']

## Housing density

In [4]:
housing_density = pd.read_excel('data/housing density.xlsx', sheet_name= 'Number of dwellings')
housing_density = housing_density.drop(index=0)
for area_london in housing_density['Area name']:
    if area_london not in areas:
        housing_density = housing_density[housing_density['Area name'] != area_london]
housing_density.rename(columns={'Area name': 'Area'}, inplace=True)
housing_density.rename(columns={'ONS code': 'Code'}, inplace=True)

# TODO: actually compute the density --> waiting for mischa's data

housing_density = housing_density[['Code','Area',2011,2012,2013,2014,2015,2016,2017]]
housing_density = pd.melt(housing_density, id_vars=['Code','Area'], var_name='Year', value_name='Number_of_dwellings')

housing_density.dropna(inplace=True)

housing_density.to_csv('clean_data/number_of_dwellings.csv',index=False)

housing_density

,Code,Area,Year,Number_of_dwellings
0,E09000001,City of London,2011,5513.000000
1,E09000002,Barking and Dagenham,2011,71079.000000
2,E09000003,Barnet,2011,139346.000000
3,E09000004,Bexley,2011,95037.000000
4,E09000005,Brent,2011,112083.000000
...,...,...,...,...
232,E09000029,Sutton,2017,82281.000000
233,E09000030,Tower Hamlets,2017,118012.129261
234,E09000031,Waltham Forest,2017,102317.000000
235,E09000032,Wandsworth,2017,143915.000000


## Crime

In [8]:
crime = pd.read_csv('data/crime.csv')
crime = crime.drop(crime.columns[[0,1]], axis=1)
crime = crime.reset_index(drop=True)
for col in crime.columns[1:]:
    if '04' in col:
        continue
    else:
        crime = crime.drop(col, axis=1)

crime = crime.groupby('LookUp_BoroughName').agg('sum').reset_index()

for area_london in crime['LookUp_BoroughName']:
    if area_london not in areas:
        crime = crime[crime['LookUp_BoroughName'] != area_london]

new_name = {}
for col in crime.columns[1:]:
    new_name[col] = col[:4]

crime.rename(columns={'LookUp_BoroughName': 'Area'}, inplace=True)
crime.rename(columns=new_name, inplace=True)
crime = crime.dropna()

# Adding missing Area Column
crime = pd.merge(housing_density[['Code', 'Area']],crime, on = 'Area')

# save clean data

crime = crime[['Code','Area','2011','2012','2013','2014','2015','2016','2017']]
crime = pd.melt(crime, id_vars=['Code','Area'], var_name='Year', value_name='Crime_rate')

crime.rename(columns = {'Crime_rate': 'Number_of_crimes' }, inplace= True)

crime.to_csv('clean_data/crime.csv',index=False)

crime

,Code,Area,Year,Number_of_crimes
0,E09000002,Barking and Dagenham,2011,1635
1,E09000003,Barnet,2011,2243
2,E09000004,Bexley,2011,982
3,E09000005,Brent,2011,2601
4,E09000006,Bromley,2011,1797
...,...,...,...,...
219,E09000029,Sutton,2017,953
220,E09000030,Tower Hamlets,2017,2727
221,E09000031,Waltham Forest,2017,1762
222,E09000032,Wandsworth,2017,1936


## Waste

In [9]:
waste = pd.read_excel('data/recycle.xlsx', sheet_name='Household Recycling Rates')
waste = waste.drop(index=0)
waste = waste.drop('Code', axis=1)
for area_london in waste['Area']:
    if area_london not in areas:
        waste = waste[waste['Area'] != area_london]
new_name = {}
for col in waste.columns[2:]:
    new_name[col] = col[:4]

waste.rename(columns={'New Code': 'Code'}, inplace=True)
waste.rename(columns=new_name).dropna()
waste.rename(columns=new_name, inplace=True)
waste = waste.dropna()

# save clean data

waste = waste[['Code','Area','2011','2012','2013','2014','2015','2016','2017']]
waste = pd.melt(waste, id_vars=['Code','Area'], var_name='Year', value_name='waste')

waste.rename(columns = {'waste': 'Waste_recycling_rates' }, inplace= True)

waste.to_csv('clean_data/waste.csv',index=False)

waste

,Code,Area,Year,Waste_recycling_rates
0,E09000001,City of London,2011,36.900000
1,E09000002,Barking and Dagenham,2011,30.000000
2,E09000003,Barnet,2011,33.600000
3,E09000004,Bexley,2011,53.500000
4,E09000005,Brent,2011,36.800000
...,...,...,...,...
226,E09000029,Sutton,2017,50.048815
227,E09000030,Tower Hamlets,2017,26.449413
228,E09000031,Waltham Forest,2017,32.481245
229,E09000032,Wandsworth,2017,22.085340
